<a href="https://colab.research.google.com/github/trentfridey/qosf-screening/blob/master/Task1FromScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import autograd.numpy as np
from autograd import grad
from scipy.linalg import kron
from functools import reduce
from autograd.misc.optimizers import adam
import matplotlib.pyplot as plt

In [38]:
# Operators

sx = np.array([[0,1],[1,0]])
sy = np.array([[0,-1j],[1j,0]])
sz = np.array([[1,0],[0,-1]])

p0 = np.array([[1,0],[0,0]])
p1 = np.array([[0,0],[0,1]])

# Gates

def Rx(theta):
  mat = -1j*theta*0.5*sx
  return np.exp(mat)

def Ry(theta):
  mat = -1j*theta*0.5*sy
  return np.exp(mat)

def Rz(theta):
  return np.array([[np.exp(-0.5j*theta), 0], [0, np.exp(0.5j*theta)]])

def CZ(i,j):
  ops0 = [np.eye(2) for i in range(4)]
  ops1 = [np.eye(2) for i in range(4)]
  ops0[i] = p0
  ops1[i] = p1
  ops1[j] = sz
  cz0 = reduce(lambda res, op: kron(res, op), ops0)
  cz1 = reduce(lambda res, op: kron(res, op), ops1)
  return cz0 + cz1

# Blocks

def odd(thetas):
  rots = [Rx(theta) for theta in thetas[1:]]
  rot = Rx(thetas[0])
  for r in rots:
    rot = kron(rot,r)
  return rot

def even(thetas):
  rots = [Rz(theta) for theta in thetas[1:]]
  rot = Rx(thetas[0])
  for r in rots:
    rot = kron(rot, r)
  CZs = np.eye(16)
  for i in range(3):
    for j in range(i+1, 4):
      CZs = CZs @ CZ(i,j)
  return rot @ CZs

# Layer

def layer(thetas):
  return odd(thetas[0:4]) @ even(thetas[4:8])

n_layers = 1

# Circuit

def circuit(thetas):
  circuit = np.eye(16)
  for l in range(n_layers):
    circuit = circuit @ layer(thetas[8*l:8*(l+1)])
  return circuit

# Objective

target = np.array([0 for i in range(0,15)] + [1])

def objective(thetas):
  start_state = [1] + [0 for i in range(1,16)]
  circ = circuit(thetas)
  return 2 - 2*np.real(np.vdot(start_state, np.dot(circuit, target)))